In [23]:
import warnings
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import Button, Layout
from datetime import datetime
# warnings.filterwarnings("ignore")
from umap import UMAP
import pandas as pd
import matplotlib.patches as mpatches
from pprint import pprint
import numpy as np
from ipywidgets import HBox, VBox, interactive
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from IPython.display import display, Markdown
import ipywidgets as widgets
import math
from sklearn.manifold import TSNE
import torch
from ipywidgets import interact, interact_manual
from IPython.display import set_matplotlib_formats
from importlib import reload
from pathlib import Path
from tqdm import tqdm

import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

import model_codebase as cb
import cicids2017 as cicids2017
import data_generator as generator

reload(cicids2017)
reload(generator)

pd.set_option('display.max_rows', 500)
pd.set_option('float_format', '{:.4f}'.format)

RES_PATH = "."
MODELPATH = Path(f"../res/{RES_PATH}/ts2vec.torch")
DATASETPATH_CACHE = Path(f"../res/{RES_PATH}/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")

In [24]:
mMIN = 50
M = 254 - mMIN
MAX_ENC = M + math.pow(M, 2) + math.pow(M, 3)

def encdec(rgb=None, i=None):
    """
    References:
        https://coderwall.com/p/fzni3g/bidirectional-translation-between-1d-and-3d-arrays
    """
    if rgb is not None:
        i = rgb[0] + rgb[1] * M + rgb[2] * math.pow(M, 2)
        return math.floor(i)
    x = math.floor(i % M)
    y = math.floor((i / M) % M)
    z = math.floor(i / ( math.pow(M, 2) ))
    return (x+mMIN, y+mMIN, z+mMIN)
           
# def ncolors(n, givehex=False):
#    cols = np.linspace(0, MAX_ENC, n)
#    cols = map(lambda x: encdec(i=x), cols)
#    
#    if givehex:
#        cols = map(lambda x: rgbtohex(*x), cols)
#    return list(cols)

def ncolors(n):
    hexl = list('0123456789ABCDEF')
    hexc = np.random.choice(hexl, size=(n, 6))
    return ['#' + ''.join(x) for x in hexc]
    
def rgbtohex(r, g, b):
    hexcol = "#{:02x}{:02x}{:02x}".format(r,g,b)
    return hexcol.upper()

### Loading model and data

In [25]:
TARGET_DSET = "validation"

ts2vec = cb.STC().eval()
ts2vec.load_state_dict(torch.load(str(MODELPATH), map_location=torch.device('cpu')))

dset = cicids2017.load_dataset(DATASETPATH_CACHE)[TARGET_DSET]
dset, _ = cicids2017.trainsplit(dset, .7)
X = torch.Tensor(dset["context"])
actv = X[:, :cb.ACTIVITY_LEN]

with torch.no_grad():    
    dset["embedding"] = ts2vec.toembedding(actv).numpy()
    
# dset["ebs2D"] = UMAP().fit_transform(dset["embedding"])
dset["ebs2D"] = TSNE(n_components=2).fit_transform(dset["embedding"])

In [26]:
timestamp2weekday = lambda x: datetime.fromtimestamp(x).weekday()
dset_day = list(map(timestamp2weekday, dset["start_time"]))
dset["weekday"] = np.array(dset_day)

# Broadcast address fix :-)
dset["device_category"][np.where(dset["host"]=="192.168.10.255")] = "broadcast"

### What about K-Means?

In [58]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=25, random_state=0).fit(dset["ebs2D"])
dset["kmeans"] = kmeans.labels_

### Widget creation

In [27]:
%matplotlib inline
set_matplotlib_formats('svg')

DAY2INT = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4}
INT2DAY = {v: k for k, v in DAY2INT.items()}

devices = set([(c, h) for (c, h) in zip(dset["device_category"], dset["host"])])
devices_str = [f"{h} ({c})" for c, h in devices]
devices_str.sort()


# ----- ----- COLORMAPS ----- ----- #
# ----- ----- ------- ----- ----- #
everythingAcolor = np.concatenate([
                        np.unique(dset["attack_type"]),
                        [dev[0] for dev in devices],
                        [dev[1] for dev in devices]])  
everythingAcolor = dict(zip(everythingAcolor, ncolors(len(everythingAcolor))))


# ----- ----- WIDGETS ----- ----- #
# ----- ----- ------- ----- ----- #
device_w_list = widgets.Dropdown(options=devices_str,
                                 value="192.168.10.51 (server)")

available_days = list(map(lambda x: INT2DAY[x], np.unique(dset["weekday"])))
days_w_list = widgets.Dropdown(options=available_days)

available_attacks = list(np.unique(dset["attack_type"]))+["All"]
show_attacks_dropdown = widgets.Dropdown(options=available_attacks, value="none")

netmap_checkbox = widgets.Checkbox(value=False, description='Show all network')
all_week = widgets.Checkbox(value=False, description='Show all week traffic')
show_category_checkbox = widgets.Checkbox(value=False, description='Show category')
all_week_checkbox = widgets.Checkbox(value=False, description='Show all week traffic data')

ts1_selector = HBox([device_w_list, days_w_list])
ts2_selector = HBox([netmap_checkbox, show_category_checkbox])
ts3_selector = HBox([all_week_checkbox, show_attacks_dropdown])
wlist = VBox([ts1_selector, ts2_selector, ts3_selector])


# ----- ----- INTERACTOR ----- ----- #
# ----- ----- ---------- ----- ----- #
def plotme(x1, x2, colors, labels):
    ax = plt.gca()
    # ax = plt.axes(projection='3d')
    for l in np.unique(labels):
        l_idxs = np.where(labels==l)[0]
        ax.scatter(x1[l_idxs], x2[l_idxs], color=colors[l_idxs], label=l)
    plt.title("t-SNE validation data, attacks excluded")
    # ax.view_init(0, 90)
    
def whandler(device, day, netmap, show_category, all_week, attack):
    if all_week:
        idx_mask = np.full(len(dset["weekday"]), True)
    else:
        idx_mask = (dset["weekday"] == DAY2INT[day])
    
    host_only = not netmap
    if host_only:
        # Extracting host/category
        split = device.split(" ")
        host = split[0].strip()
        category = " ".join(split[1:]).replace("(", "").replace(")", "").strip()
        # Masking
        host_mask = (dset["host"]==host)
        idx_mask = idx_mask & host_mask
    masked_dset = { k: v[idx_mask] for k, v in dset.items() }
    
    x1 = masked_dset["ebs2D"][:, 0]
    x2 = masked_dset["ebs2D"][:, 1]
    # z = masked_dset["kmeans"]

    focus_label = "device_category" if netmap and show_category else "host"
    colors = np.array(list(map(lambda x: everythingAcolor[x], masked_dset[focus_label].squeeze())))
    labels = masked_dset[focus_label]
    plotme(x1, x2, colors, labels)
    
    # Plotting attacks..... #
    if attack != "none":
        if attack=="all":
            attack_mask = masked_dset["attack_type"] != "none"
        else:
            attack_mask = masked_dset["attack_type"] == attack
        attack_masked_dset = masked_dset["attack_type"][attack_mask].squeeze()    
        colors = np.array(list(map(lambda x: everythingAcolor[x], attack_masked_dset)))
        labels = masked_dset["attack_type"][attack_mask]
        plotme(x1, x2, colors, labels)

    # plt.legend(loc=(1.04,0))
    plt.legend()
    plt.gcf().set_size_inches(7, 7)
    plt.savefig("tSNEclustering.png")
    # plt.show()

In [28]:
output = widgets.interactive(whandler,
                             all_week=all_week_checkbox,
                             device=device_w_list, day=days_w_list, 
                             netmap=netmap_checkbox, 
                             show_category=show_category_checkbox,
                             attack=show_attacks_dropdown).children[-1]
display(wlist)
display(output)

Output()